In [27]:
import pickle
import numpy as np
import folium

In [2]:
data = pickle.load(open('./pickles/realtor_clean.p','rb'))
residual_model = pickle.load(open('./pickles/final_model.p','rb'))
location_model = pickle.load(open('./pickles/location_model.p','rb'))

In [6]:
data[(data['BsmtQual_Ex'] == 1) & (data['ExterQual_Ex'] == 1)]

,GrLivArea,LotArea,OverallQual,OverallCond,TotalBsmtSF,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,PoolArea,latitude,...,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_Po,KitchenQual_TA,Quarter_Q1,Quarter_Q2,Quarter_Q3,Quarter_Q4,Location_Price,Residuals
710,1203.0,12003.0,5,4,1176.0,3.0,6.0,1.0,0.0,42.056932,...,0,0,0,1,0,1,0,0,178780.0,121350.538814
959,2977.0,25500.0,4,6,2491.0,4.0,10.0,0.0,0.0,42.059253,...,0,0,0,1,0,1,0,0,268258.7,232738.198785
2879,2494.0,12073.0,7,5,1362.0,3.0,8.0,1.0,0.0,42.053440,...,1,0,0,0,0,1,0,0,350275.0,200043.071486
3419,1712.0,7218.0,4,3,840.0,3.0,10.0,1.0,0.0,42.052273,...,0,0,0,1,0,1,0,0,279892.5,122560.945803
4290,1824.0,13500.0,6,8,0.0,4.0,10.0,2.0,0.0,42.035114,...,0,0,0,1,0,1,0,0,175590.0,146496.166094
5242,2899.0,25502.0,7,5,1475.0,4.0,8.0,1.0,0.0,42.037190,...,0,0,0,1,0,1,0,0,228500.0,219686.542366
5873,1068.0,8260.0,5,6,588.0,2.0,5.0,0.0,0.0,42.043729,...,0,0,0,1,0,1,0,0,134418.3,106964.403215
6493,1368.0,12850.0,7,7,1252.0,3.0,6.0,1.0,0.0,42.048842,...,0,0,0,1,0,1,0,0,202880.0,159585.273355
7112,1553.0,16050.0,7,5,1553.0,2.0,6.0,0.0,0.0,42.034853,...,0,1,0,0,0,1,0,0,130390.0,175308.345313
7974,849.0,10800.0,6,5,849.0,1.0,4.0,0.0,0.0,42.033633,...,0,1,0,0,0,1,0,0,124090.0,93738.299311


In [55]:
test_old = data.drop(['Residuals'], axis = 1).iloc[np.random.randint(data.shape[0]),:].to_frame().transpose()

In [71]:
test_new = test_old.copy()

In [69]:
location_model.predict(test_old[['latitude','e_long']]) + residual_model.predict(test_old.drop(['e_long','Location_Price','latitude','longitude'],axis = 1))

array([133795.17518706])

In [73]:
location_model.predict(test_old[['latitude','e_long']]) + residual_model.predict(test_new.drop(['e_long','Location_Price','latitude','longitude'],axis = 1))

array([255502.11463543])

In [76]:
test_new.drop(['e_long','latitude','longitude'],axis = 1)[test_new != 0].dropna(axis = 1).transpose()

,15825
GrLivArea,864.0
LotArea,8212.0
OverallQual,6.0
OverallCond,5.0
TotalBsmtSF,864.0
BedroomAbvGr,2.0
TotRmsAbvGrd,5.0
LandSlope_Gtl,1.0
BldgType_1Fam,1.0
HouseStyle_1Story,1.0


In [72]:
test_new['Quarter_Q2'] = 0; test_new['Quarter_Q3'] = 1
test_new['BsmtQual_Gd'] = 0; test_new['BsmtQual_Ex'] = 1
test_new['Age'] = 0
test_new['RoofMatl_CompShg'] = 0; test_new['RoofMatl_WdShngl'] = 1
test_new['ExterQual_TA'] = 0; test_new['ExterQual_Ex'] = 1
test_new['CentralAir_N'] = 0; test_new['CentralAir_Y'] = 1
residual_model.predict(test_new.drop(['latitude','longitude'],axis = 1)) - residual_model.predict(test_old.drop(['latitude','longitude'],axis = 1))

In [78]:
data.shape[0]

8901

In [95]:
row = data.iloc[0,:]
row.loc[[col for col in row.index if 'ExterQual' in col]] =  0

<ipython-input-95-0a5fb16e5a17>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row.loc[[col for col in row.index if 'ExterQual' in col]] =  0


In [104]:
residual_model.predict(row.drop(['Location_Price','longitude','latitude','e_long','Residuals']).to_frame().transpose())

array([76366.43796615])

In [109]:
def potential(row):
    new_row = row.copy()
    new_row.Age = 0
    new_row.Quarter_Q2 = 0; new_row.Quarter_Q3 = 1
    #ExterQual
    new_row.loc[[col for col in new_row.index if 'ExterQual' in col]] =  0
    new_row.ExterQual_Ex = 1
    #Kitchenn
    new_row.loc[[col for col in new_row.index if 'Kitchen' in col]] =  0
    new_row.KitchenQual_Ex = 1
    #Bsmt
    new_row.loc[[col for col in new_row.index if 'BsmtQual' in col]] =  0
    new_row.BsmtQual_Ex = 1
    #Central_Air
    new_row.CentralAir_N = 0
    new_row.CentralAir_Y = 1
    #Roof
    new_row.loc[[col for col in new_row.index if 'Roof' in col]] =  0
    new_row.RoofMatl_WdShngl = 1
    #model
    potential_ = residual_model.predict(new_row.drop(['Location_Price','longitude','latitude','e_long','Residuals']).to_frame().transpose()) - residual_model.predict(row.drop(['Location_Price','longitude','latitude','e_long','Residuals']).to_frame().transpose())
    return(potential_)

In [113]:
data_w_potential = data.copy()
data_w_potential['potential'] = data.apply(potential,axis = 1)
data_w_potential.loc[:,'potential'] = data_w_potential.loc[:,'potential'].apply(lambda x: x[0])

In [128]:
data_most_potential = data_w_potential.sort_values(by = 'potential')[-20:]

In [130]:
sum(data_most_potential.potential)

2751033.4469959675

In [129]:
m = folium.Map(location=[test_old.latitude,test_old.longitude],tiles='Stamen Toner')
for i in range(0,data_most_potential.shape[0]):
    folium.Marker(location = [data_most_potential.latitude.iloc[i], data_most_potential.longitude.iloc[i]]).add_to(m)
m

In [77]:
m = folium.Map(location=[test_old.latitude,test_old.longitude],tiles='Stamen Toner')
folium.Marker([test_old.latitude,test_old.longitude],popup = 'Example House').add_to(m)
m